# Configuration

In [2]:
import os
from tqdm.notebook import tqdm
from tqdm import tqdm
import librosa
import pandas as pd 
import pandas as np
from sklearn.metrics import classification_report

MODELS_FOLDER = "models"
SCALERS_FOLDER = "scalers"

# METTERE PATH MODELS E SCLAERS E CALCOLARE LA FINAL PREDICTION DIVERSAMENTE PRENDENDENDO PEZZI DI CODICE DELL'ENSEMBLE

In [3]:
# import utils
import feature_extraction as fe
import emotion_predictions as ep
import scipy
import numpy as np
from scipy import signal
from scipy.io.wavfile import write
#from utils import resample, denoise
import sklearn
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
# print('The scikit-learn version is {}.'.format(sklearn.__version__))

# Clean Data - Compute dataframes for datasets and split in Train, Val, Test 

In [4]:
main_path = '/Users/helemanc/Documents/MasterAI/THESIS/Datasets SER'
TESS = os.path.join(main_path, "tess/TESS Toronto emotional speech set data/") 
RAV = os.path.join(main_path, "ravdess-emotional-speech-audio/audio_speech_actors_01-24")
SAVEE = os.path.join(main_path, "savee/ALL/")
CREMA = os.path.join(main_path, "creamd/AudioWAV/")

## RAVDESS

In [5]:
lst = []
emotion = []
voc_channel = []
full_path = []
modality = []
intensity = []
actors = []
phrase =[]

for root, dirs, files in tqdm(os.walk(RAV)):
    for file in files:
        try:
            #Load librosa array, obtain mfcss, store the file and the mfcss information in a new array
            # X, sample_rate = librosa.load(os.path.join(root,file), res_type='kaiser_fast')
            # mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
            # The instruction below converts the labels (from 1 to 8) to a series from 0 to 7
            # This is because our predictor needs to start from 0 otherwise it will try to predict also 0.
           
            modal = int(file[1:2])
            vchan = int(file[4:5])
            lab = int(file[7:8])
            ints = int(file[10:11])
            phr = int(file[13:14])
            act = int(file[18:20])
            # arr = mfccs, lab
            # lst.append(arr)
            
            modality.append(modal)
            voc_channel.append(vchan)
            emotion.append(lab) #only labels
            intensity.append(ints)
            phrase.append(phr)
            actors.append(act)
            
            full_path.append((root, file)) # only files
          # If the file is not valid, skip it
        except ValueError:
            continue

25it [00:00, 268.86it/s]


In [6]:
# 01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised
# merge neutral and calm
emotions_list = ['neutral', 'neutral', 'happy', 'sadness', 'angry', 'fear', 'disgust', 'surprise']
emotion_dict = {em[0]+1:em[1] for em in enumerate(emotions_list)}

df = pd.DataFrame([emotion, voc_channel, modality, intensity, actors, actors,phrase, full_path]).T
df.columns = ['emotion', 'voc_channel', 'modality', 'intensity', 'actors', 'gender', 'phrase', 'path']
df['emotion'] = df['emotion'].map(emotion_dict)
df['voc_channel'] = df['voc_channel'].map({1: 'speech', 2:'song'})
df['modality'] = df['modality'].map({1: 'full AV', 2:'video only', 3:'audio only'})
df['intensity'] = df['intensity'].map({1: 'normal', 2:'strong'})
df['actors'] = df['actors']
df['gender'] = df['actors'].apply(lambda x: 'female' if x%2 == 0 else 'male')
df['phrase'] = df['phrase'].map({1: 'Kids are talking by the door', 2:'Dogs are sitting by the door'})
df['path'] = df['path'].apply(lambda x: x[0] + '/' + x[1])

In [7]:
# remove files with noise to apply the same noise to all files for data augmentation 
df = df[~df.path.str.contains('noise')]

In [8]:
df.head()

,emotion,voc_channel,modality,intensity,actors,gender,phrase,path
0,angry,speech,audio only,normal,16,female,Dogs are sitting by the door,/Users/helemanc/Documents/MasterAI/THESIS/Data...
1,fear,speech,audio only,normal,16,female,Dogs are sitting by the door,/Users/helemanc/Documents/MasterAI/THESIS/Data...
2,fear,speech,audio only,strong,16,female,Kids are talking by the door,/Users/helemanc/Documents/MasterAI/THESIS/Data...
3,angry,speech,audio only,strong,16,female,Kids are talking by the door,/Users/helemanc/Documents/MasterAI/THESIS/Data...
5,disgust,speech,audio only,normal,16,female,Kids are talking by the door,/Users/helemanc/Documents/MasterAI/THESIS/Data...


In [9]:
# only speech
RAV_df = df
RAV_df = RAV_df.loc[RAV_df.voc_channel == 'speech']

In [10]:
RAV_df.insert(0, "emotion_label", RAV_df.emotion, True)

In [11]:
RAV_df = RAV_df.drop(['emotion', 'voc_channel', 'modality', 'intensity', 'phrase'], 1)

In [12]:
RAV_df

,emotion_label,actors,gender,path
0,angry,16,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...
1,fear,16,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...
2,fear,16,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...
3,angry,16,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...
5,disgust,16,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...
...,...,...,...,...
2869,happy,8,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...
2871,happy,8,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...
2872,neutral,8,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...
2873,neutral,8,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...


In [13]:
RAV_train = []
RAV_val = []
RAV_test = []

In [14]:
for index, row in RAV_df.iterrows():
    if row['actors'] in range(1,21): 
        RAV_train.append(row) 
    elif row['actors'] in range(21,23): 
        RAV_val.append(row)
    elif row['actors'] in range(23,25): 
        RAV_test.append(row)
len(RAV_train), len(RAV_val), len(RAV_test)

(1200, 120, 120)

In [15]:
RAV_train = pd.DataFrame(RAV_train)
RAV_val = pd.DataFrame(RAV_val)
RAV_test = pd.DataFrame(RAV_test)

In [16]:
RAV_train = RAV_train.drop(['actors'], 1)
RAV_val = RAV_val.drop(['actors'], 1)
RAV_test = RAV_test.drop(['actors'], 1)

In [17]:
RAV_train.reset_index(drop=True, inplace = True) 
RAV_val.reset_index(drop=True, inplace = True) 
RAV_test.reset_index(drop=True, inplace = True ) 

## SAVEE

In [18]:
# Get the data location for SAVEE
dir_list = os.listdir(SAVEE)

# parse the filename to get the emotions
emotion=[]
path = []
actors = []
gender = []
for i in dir_list:
    actors.append(i[:2])
    if i[-8:-6]=='_a':
        emotion.append('angry')
        gender.append('male')
    elif i[-8:-6]=='_d':
        emotion.append('disgust')
        gender.append('male')
    elif i[-8:-6]=='_f':
        emotion.append('fear')
        gender.append('male')
    elif i[-8:-6]=='_h':
        emotion.append('happy')
        gender.append('male')
    elif i[-8:-6]=='_n':
        emotion.append('neutral')
        gender.append('male')
    elif i[-8:-6]=='sa':
        emotion.append('sadness')
        gender.append('male')
    elif i[-8:-6]=='su':
        emotion.append('surprise')
        gender.append('male') 
    else:
        emotion.append('Unknown') 
    path.append(SAVEE + i)
    
# Now check out the label count distribution 
SAVEE_df = pd.DataFrame(emotion, columns = ['emotion_label'])
                      
SAVEE_df = pd.concat([SAVEE_df,
                      pd.DataFrame(actors, columns = ['actors']),
                      pd.DataFrame(gender, columns = ['gender']), 
                      pd.DataFrame(path, columns = ['path'])], axis = 1)
SAVEE_df.emotion_label.value_counts()

neutral     120
happy        60
angry        60
sadness      60
disgust      60
fear         60
surprise     60
Name: emotion_label, dtype: int64

In [19]:
SAVEE_df.head()

,emotion_label,actors,gender,path
0,sadness,JK,male,/Users/helemanc/Documents/MasterAI/THESIS/Data...
1,sadness,JK,male,/Users/helemanc/Documents/MasterAI/THESIS/Data...
2,neutral,DC,male,/Users/helemanc/Documents/MasterAI/THESIS/Data...
3,surprise,DC,male,/Users/helemanc/Documents/MasterAI/THESIS/Data...
4,neutral,DC,male,/Users/helemanc/Documents/MasterAI/THESIS/Data...


In [20]:
SAVEE_train = []
SAVEE_val = []
SAVEE_test = []

In [21]:
#DC, JE, JK, KL
for index, row in SAVEE_df.iterrows(): 
    if row['actors'] == 'DC' or row ['actors'] == 'JE':
        SAVEE_train.append(row)
    elif row['actors'] == 'JK': 
        SAVEE_val.append(row)
    else: 
        SAVEE_test.append(row)
len(SAVEE_train), len(SAVEE_val), len(SAVEE_test)

(240, 120, 120)

In [22]:
SAVEE_train = pd.DataFrame(SAVEE_train)
SAVEE_val = pd.DataFrame(SAVEE_val)
SAVEE_test = pd.DataFrame(SAVEE_test)

In [23]:
SAVEE_train = SAVEE_train.drop(['actors'], 1)
SAVEE_val = SAVEE_val.drop(['actors'], 1)
SAVEE_test = SAVEE_test.drop(['actors'], 1)

In [24]:
SAVEE_train = SAVEE_train.reset_index(drop=True) 
SAVEE_val = SAVEE_val.reset_index(drop=True) 
SAVEE_test = SAVEE_test.reset_index(drop=True) 

## TESS

In [25]:
dir_list = os.listdir(TESS)
dir_list.sort()
dir_list

path = []
emotion = []
gender = []
actors = []

for i in dir_list:
    fname = os.listdir(TESS + i)
    for f in fname:
        if i == 'OAF_angry':
            emotion.append('angry')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_angry': 
            emotion.append('angry')
            gender.append('female')
            actors.append('YAF')
            
            
        elif i == 'OAF_disgust' :
            emotion.append('disgust')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_disgust': 
            emotion.append('disgust')
            gender.append('female')
            actors.append('YAF')
            
            
        elif i == 'OAF_Fear':
            emotion.append('fear')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_fear': 
            emotion.append('fear')
            gender.append('female')
            actors.append('YAF') 
            
            
        elif i == 'OAF_happy' :
            emotion.append('happy')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_happy': 
            emotion.append('angry')
            gender.append('female')
            actors.append('YAF')            
            
        elif i == 'OAF_neutral':
            emotion.append('neutral')
            gender.append('female')
            actors.append('OAF')   
        elif i == 'YAF_neutral': 
            emotion.append('neutral')
            gender.append('female')
            actors.append('YAF')      
            
                
        elif i == 'OAF_Pleasant_surprise':
            emotion.append('surprise')
            gender.append('female')
            actors.append('OAF')
        
        elif i == 'YAF_pleasant_surprised': 
            emotion.append('surprise')
            gender.append('female')
            actors.append('YAF')            
            
        elif i == 'OAF_Sad':
            emotion.append('sadness')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_sad': 
            emotion.append('sadness')
            gender.append('female')
            actors.append('YAF')            
        else:
            emotion.append('Unknown')
        path.append(TESS + i + "/" + f)

TESS_df = pd.DataFrame(emotion, columns = ['emotion_label'])
TESS_df = pd.concat([TESS_df, pd.DataFrame(gender, columns = ['gender']), 
                     pd.DataFrame(actors, columns= ['actors']),
                     pd.DataFrame(path, columns = ['path'])],axis=1)
TESS_df.emotion_label.value_counts()

angry       1200
fear         800
neutral      800
surprise     800
sadness      800
disgust      800
happy        400
Name: emotion_label, dtype: int64

In [26]:
TESS_df= TESS_df[~TESS_df.path.str.contains('noise')]

In [27]:
TESS_train = []
TESS_test = []

In [28]:
for index, row in TESS_df.iterrows(): 
    if row['actors'] == 'YAF': 
        TESS_train.append(row)
    else: 
        TESS_test.append(row)
len(TESS_train), len(TESS_test)

(1400, 1400)

In [29]:
TESS_train = pd.DataFrame(TESS_train)
TESS_test = pd.DataFrame(TESS_test)

In [30]:
TESS_train = TESS_train.drop(['actors'], 1)
TESS_test = TESS_test.drop(['actors'], 1)

In [31]:
TESS_train = TESS_train.reset_index(drop=True) 
TESS_test  = TESS_test.reset_index(drop=True) 

## CREMA-D

In [32]:
males = [1,
5,
11,
14,
15,
16,
17,
19,
22,
23,
26,
27,
31,
32,
33,
34,
35,
36,
38,
39,
41,
42,
44,
45,
48,
50,
51,
57,
59, 
62, 
64,
65, 
66,
67,
68,
69,
70,
71,
77, 
80, 
81, 
83, 
85, 
86, 
87,
88, 
90]

In [33]:
females = [ 2,
3,
4,
6,
7,
8,
9,
10,
12,
13,
18,
20,
21,
24,
25,
28,
29,
30,
37,
40,
43,
46,
47,
49,
52,
53,
54,
55,
56, 
58, 
60,
61,
63,
72, 
73, 
74, 
75, 
76, 
78, 
79, 
82, 
84, 
89, 
91]

In [34]:
crema_directory_list = os.listdir(CREMA)

file_emotion = []
file_path = []
actors = []
gender = []




for file in crema_directory_list:

    # storing file emotions
    part=file.split('_')
    
    # use only high intensity files
    if "HI" in part[3] :
        actor = part[0][2:]
        actors.append(actor)
        if int(actor) in males:
            gender.append('male')
        else: 
            gender.append('female')
    
        # storing file paths
        file_path.append(CREMA + file)
        if part[2] == 'SAD':
            file_emotion.append('sadness')
        elif part[2] == 'ANG':
            file_emotion.append('angry')
        elif part[2] == 'DIS':
            file_emotion.append('disgust')
        elif part[2] == 'FEA':
            file_emotion.append('fear')
        elif part[2] == 'HAP':
            file_emotion.append('happy')
        elif part[2] == 'NEU':
            file_emotion.append('neutral')
        else:
            file_emotion.append('Unknown')

# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['emotion_label'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['path'])
actors_df = pd.DataFrame(actors, columns=['actors'])
gender_df = pd.DataFrame(gender, columns=['gender'])                      
Crema_df = pd.concat([emotion_df, actors_df, gender_df, path_df], axis=1)
Crema_df.head()

,emotion_label,actors,gender,path
0,sadness,28,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...
1,angry,48,male,/Users/helemanc/Documents/MasterAI/THESIS/Data...
2,disgust,27,male,/Users/helemanc/Documents/MasterAI/THESIS/Data...
3,disgust,32,male,/Users/helemanc/Documents/MasterAI/THESIS/Data...
4,happy,80,male,/Users/helemanc/Documents/MasterAI/THESIS/Data...


In [35]:
Crema_df.shape

(455, 4)

In [36]:
actor_files = {}

for index, row in Crema_df.iterrows():
    actor = row['actors']
    if actor not in actor_files.keys(): 
        actor_files[actor] = 1
    else: 
        actor_files[actor]+=1

In [37]:
actor_files

{'28': 5,
 '48': 5,
 '27': 5,
 '32': 5,
 '80': 5,
 '59': 5,
 '39': 5,
 '43': 5,
 '56': 5,
 '31': 5,
 '24': 5,
 '51': 5,
 '44': 5,
 '77': 5,
 '62': 5,
 '88': 5,
 '55': 5,
 '40': 5,
 '13': 5,
 '06': 5,
 '35': 5,
 '20': 5,
 '87': 5,
 '09': 5,
 '78': 5,
 '65': 5,
 '70': 5,
 '23': 5,
 '36': 5,
 '01': 5,
 '14': 5,
 '47': 5,
 '52': 5,
 '58': 5,
 '37': 5,
 '22': 5,
 '71': 5,
 '64': 5,
 '50': 5,
 '45': 5,
 '57': 5,
 '42': 5,
 '29': 5,
 '53': 5,
 '46': 5,
 '33': 5,
 '26': 5,
 '34': 5,
 '21': 5,
 '15': 5,
 '08': 5,
 '86': 5,
 '79': 5,
 '63': 5,
 '76': 5,
 '25': 5,
 '30': 5,
 '07': 5,
 '12': 5,
 '41': 5,
 '54': 5,
 '89': 5,
 '81': 5,
 '49': 5,
 '38': 5,
 '05': 5,
 '10': 5,
 '61': 5,
 '74': 5,
 '69': 5,
 '18': 5,
 '83': 5,
 '17': 5,
 '02': 5,
 '73': 5,
 '66': 5,
 '84': 5,
 '91': 5,
 '90': 5,
 '85': 5,
 '03': 5,
 '16': 5,
 '67': 5,
 '72': 5,
 '68': 5,
 '82': 5,
 '19': 5,
 '11': 5,
 '04': 5,
 '75': 5,
 '60': 5}

In [38]:
count_males = 0 
count_females = 0 
male_list = []
for index, row in Crema_df.iterrows(): 
    gender = row['gender']
    actor = row['actors']
    if gender == 'male':
        count_males +=1
        if actor not in male_list: 
            male_list.append(actor)
    else: 
        count_females +=1

In [39]:
count_males, count_females

(235, 220)

Since there are more males than females we will remove randomly 3 male actors (since there are exactly 5 audio files per actor)

In [40]:
import random 
'''
random.seed(42)
males_to_remove = random.sample(male_list, 3)
males_to_remove
'''
males_to_remove = ['17', '80', '88']

In [41]:
new_df = []
for index, row in Crema_df.iterrows(): 
    if row['actors'] not in males_to_remove: 
        new_df.append(row)

In [42]:
CREMA_df = pd.DataFrame(new_df)

In [43]:
for index, row in CREMA_df.iterrows(): 
    if row['actors'] == '17': 
        print("Elements not removed")

In [44]:
count_males = 0 
count_females = 0 
male_list = []
female_list = []
for index, row in CREMA_df.iterrows(): 
    gender = row['gender']
    actor = row['actors']
    if gender == 'male':
        count_males +=1
        if actor not in male_list: 
            male_list.append(actor)
    else: 
        count_females +=1
        if actor not in female_list: 
            female_list.append(actor)

In [45]:
count_males, count_females

(220, 220)

In [46]:
len(female_list)

44

In [47]:
len(male_list)

44

In [48]:
CREMA_train = []
CREMA_val = []
CREMA_test = []

In [49]:
females_train = random.sample(female_list, 32)
males_train = random.sample(male_list, 32)

# remove the elements assigned to train 
for element in females_train:
    if element in female_list:
        female_list.remove(element)
        
for element in males_train:
    if element in male_list:
        male_list.remove(element)

         
females_val = random.sample(female_list, 6) 
males_val = random.sample(male_list, 6) 

# remove the elements assigned to val
for element in females_val:
    if element in female_list:
        female_list.remove(element)
        
for element in males_val:
    if element in male_list:
        male_list.remove(element)
        
females_test = random.sample(female_list, 6) 
males_test = random.sample(male_list, 6)        

In [50]:
females_train, males_train, females_val, males_val, females_test, males_test

(['21',
  '37',
  '91',
  '58',
  '30',
  '46',
  '10',
  '56',
  '03',
  '78',
  '12',
  '82',
  '04',
  '72',
  '84',
  '07',
  '02',
  '29',
  '55',
  '06',
  '13',
  '20',
  '53',
  '24',
  '49',
  '28',
  '89',
  '54',
  '09',
  '60',
  '08',
  '40'],
 ['51',
  '23',
  '71',
  '50',
  '35',
  '14',
  '11',
  '59',
  '83',
  '44',
  '81',
  '45',
  '65',
  '90',
  '86',
  '22',
  '27',
  '66',
  '36',
  '05',
  '34',
  '41',
  '38',
  '67',
  '48',
  '15',
  '16',
  '85',
  '64',
  '39',
  '62',
  '77'],
 ['52', '63', '25', '73', '18', '74'],
 ['31', '68', '19', '87', '69', '57'],
 ['47', '79', '76', '61', '43', '75'],
 ['42', '33', '26', '70', '01', '32'])

In [51]:
train = females_train + males_train 
val = females_val + males_val 
test = females_test + males_test

In [52]:
for index, row in CREMA_df.iterrows(): 
    gender = row['gender']
    actor = row['actors']
    if actor in train: 
        CREMA_train.append(row)
    elif actor in val: 
        CREMA_val.append(row)
    else:
        CREMA_test.append(row)

In [53]:
CREMA_train = pd.DataFrame(CREMA_train) 
CREMA_val = pd.DataFrame(CREMA_val) 
CREMA_test = pd.DataFrame(CREMA_test)

In [54]:
CREMA_train.shape, CREMA_val.shape, CREMA_test.shape

((320, 4), (60, 4), (60, 4))

In [55]:
CREMA_train = CREMA_train.drop(['actors'], 1)
CREMA_val = CREMA_val.drop(['actors'], 1)
CREMA_test = CREMA_test.drop(['actors'], 1)

In [56]:
CREMA_train = CREMA_train.reset_index(drop=True) 
CREMA_val = CREMA_val.reset_index(drop = True) 
CREMA_test = CREMA_test.reset_index(drop = True)

# Utils 

In [57]:
def denoise(samples):
    """
    :param samples: an array representing the sampled audio file
    :type samples: float
    :return: an array representing the clean audio file
    :rtype: float
    """
    samples_wiener = scipy.signal.wiener(samples)
    return samples_wiener

def resample(input_data, sample_rate, required_sample_rate=16000, amplify=False):
    """
    Resampling function. Takes an audio with sample_rate and resamples
    it to required_sample_rate. Optionally, amplifies volume.
    ​
    :param input_data: Input audio data
    :type input_data: numpy.ndarray
    :param sample_rate: Sample rate of input audio data
    :type sample_rate: int
    :param required_sample_rate: Sample rate to resample original audio, defaults to 16000
    :type required_sample_rate: int, optional
    :param amplify: Whether to amplify audio volume, defaults to False
    :type amplify: bool, optional
    :return: Resampled audio and new sample rate
    :rtype: numpy.ndarray, int
    """
    if sample_rate < required_sample_rate:
        resampling_factor = int(round(required_sample_rate/sample_rate, 0))
        new_rate = sample_rate * resampling_factor
        samples = len(input_data) * resampling_factor
        resampled = signal.resample(input_data, samples)
    elif sample_rate > required_sample_rate:
        resampling_factor = int(round(sample_rate/required_sample_rate, 0))
        new_rate = int(sample_rate / resampling_factor)
        resampled = signal.decimate(input_data, resampling_factor)
    else:
        resampling_factor = 1
        new_rate = sample_rate
        resampled = input_data
    if amplify and input_data.size > 0:
        absolute_values = np.absolute(resampled)
        max_value = np.amax(absolute_values)
        max_range = np.iinfo(np.int16).max
        amplify_factor = max_range/max_value
        resampled = resampled * amplify_factor
        resampled = resampled.round()
    return resampled.astype(np.float32), new_rate


def make_predictions(dataset, labels, model_name): 
    predictions = []
    model_predictions_list = []
    counter = 0
    for filepath in tqdm(dataset['path']):
        samples, sample_rate = fe.read_file(filepath)
        samples, sample_rate = resample(samples, sample_rate)
        new_samples = fe.cut_pad(samples)
        #new_filepath = "tmp.wav"
        for dirpath, dirnames, filenames in os.walk(MODELS_FOLDER):
            # iterate over the list of dirnames to load the convmodel
            # iterate over the list of filenames to get the svm model
            for model in dirnames:
                model_type = 'conv'
                parts = model.split('_')
                num_exp = parts[1]
                num_data = parts[2].split('.')[0]
                id_exp = num_exp + '_' + num_data
                if id_exp == model_name: 
                    print(id_exp)
                    model_path = os.path.join(dirpath, model)
                    conv_model = tf.keras.models.load_model(model_path)
                    for scaler in os.listdir(SCALERS_FOLDER):
                        parts = scaler.split('_')
                        num_exp = parts[1]
                        num_data = parts[2].split('.')[0]
                        id_exp_scaler = num_exp + '_' + num_data
                        if id_exp == id_exp_scaler:
                            # print("Loading scaler: ", scaler)
                            scaler_conv_model = scaler
                            mfccs = fe.mfccs_scaled(new_samples, scaler_conv_model, id_exp)
                            pred = ep.make_predictions(conv_model, model_type, mfccs, 'majority')
                            # add result to list and dictionary
                            predictions.append(pred)
                            #model_predictions[id_exp] = pred
            break
        #final_prediction, model_predictions = ensemble.ensemble(new_samples, prediction_scheme, return_model_predictions = True)

        #predictions.append(final_prediction)
        #model_predictions_list.append(model_predictions) 
        print("True label", labels[counter], "Predicted label", predictions[counter])
        counter+=1
    return predictions

# Model 2.10

## Test Set RAVDESS

In [58]:
df_test = RAV_test

In [59]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)

In [60]:
predictions = make_predictions(df_test, labels,'2_10')

  0%|          | 0/120 [00:00<?, ?it/s]

2_10


  1%|          | 1/120 [00:01<03:36,  1.82s/it]

True label 1 Predicted label 1
2_10


  2%|▏         | 2/120 [00:02<03:10,  1.61s/it]

...

True label 1 Predicted label 1
2_10


 99%|█████████▉| 119/120 [02:11<00:00,  1.21it/s]

True label 0 Predicted label 0
2_10


100%|██████████| 120/120 [02:12<00:00,  1.11s/it]

True label 0 Predicted label 0


In [61]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [62]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [63]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.77      0.95      0.85        56
           1       0.94      0.75      0.83        64

    accuracy                           0.84       120
   macro avg       0.85      0.85      0.84       120
weighted avg       0.86      0.84      0.84       120



## Test Set SAVEE

In [64]:
df_test = SAVEE_test

In [65]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)

In [66]:
predictions = make_predictions(df_test, labels,'2_10')

  0%|          | 0/120 [00:00<?, ?it/s]

2_10


  1%|          | 1/120 [00:02<05:11,  2.62s/it]

True label 1 Predicted label 0
2_10


  2%|▏         | 2/120 [00:03<04:23,  2.24s/it]

...

True label 1 Predicted label 0
2_10


 99%|█████████▉| 119/120 [02:54<00:02,  2.37s/it]

True label 1 Predicted label 0
2_10


100%|██████████| 120/120 [02:56<00:00,  1.47s/it]

True label 1 Predicted label 0


In [67]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [68]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [69]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.53      0.82      0.64        60
           1       0.61      0.28      0.39        60

    accuracy                           0.55       120
   macro avg       0.57      0.55      0.52       120
weighted avg       0.57      0.55      0.52       120



## Test Set TESS

In [70]:
df_test = TESS_test

In [71]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)

In [72]:
predictions = make_predictions(df_test, labels,'2_10')

  0%|          | 0/1400 [00:00<?, ?it/s]

2_10


  0%|          | 1/1400 [00:01<36:55,  1.58s/it]

True label 1 Predicted label 0
2_10


  0%|          | 2/1400 [00:03<36:39,  1.57s/it]

...

True label 0 Predicted label 0
2_10


100%|█████████▉| 1399/1400 [27:09<00:00,  1.07it/s]

True label 0 Predicted label 1
2_10


100%|██████████| 1400/1400 [27:11<00:00,  1.17s/it]

True label 0 Predicted label 0


In [73]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [74]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [75]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.55      0.36      0.44       600
           1       0.62      0.77      0.69       800

    accuracy                           0.60      1400
   macro avg       0.58      0.57      0.56      1400
weighted avg       0.59      0.60      0.58      1400



## Test Set CREMA

In [76]:
df_test = CREMA_test

In [77]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)

In [78]:
predictions = make_predictions(df_test, labels,'2_10')

  0%|          | 0/60 [00:00<?, ?it/s]

2_10


  2%|▏         | 1/60 [00:01<01:05,  1.10s/it]

True label 1 Predicted label 1
2_10


  3%|▎         | 2/60 [00:01<01:00,  1.04s/it]

...

True label 0 Predicted label 0
2_10


 98%|█████████▊| 59/60 [01:28<00:02,  2.10s/it]

True label 1 Predicted label 1
2_10


100%|██████████| 60/60 [01:30<00:00,  1.50s/it]

True label 1 Predicted label 1


In [79]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [80]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [81]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.23      0.67      0.34        12
           1       0.84      0.44      0.58        48

    accuracy                           0.48        60
   macro avg       0.53      0.55      0.46        60
weighted avg       0.72      0.48      0.53        60



# Model 2.12

## Test Set RAVDESS

In [82]:
df_test = RAV_test

In [83]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)

In [84]:
predictions = make_predictions(df_test, labels,'2_12')

  0%|          | 0/120 [00:00<?, ?it/s]

2_12


  1%|          | 1/120 [00:01<02:05,  1.06s/it]

True label 1 Predicted label 1
2_12


  2%|▏         | 2/120 [00:02<02:04,  1.06s/it]

...

True label 1 Predicted label 1
2_12


 99%|█████████▉| 119/120 [02:06<00:00,  1.11it/s]

True label 0 Predicted label 0
2_12


100%|██████████| 120/120 [02:07<00:00,  1.06s/it]

True label 0 Predicted label 1


In [85]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [86]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [87]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.70      0.93      0.80        56
           1       0.91      0.66      0.76        64

    accuracy                           0.78       120
   macro avg       0.81      0.79      0.78       120
weighted avg       0.81      0.78      0.78       120



## Test Set SAVEE

In [88]:
df_test = SAVEE_test

In [89]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)

In [90]:
predictions = make_predictions(df_test, labels,'2_12')

  0%|          | 0/120 [00:00<?, ?it/s]

2_12


  1%|          | 1/120 [00:00<01:43,  1.15it/s]

True label 1 Predicted label 1
2_12


  2%|▏         | 2/120 [00:01<01:40,  1.18it/s]

...

True label 1 Predicted label 1
2_12


 99%|█████████▉| 119/120 [01:59<00:00,  1.15it/s]

True label 1 Predicted label 0
2_12


100%|██████████| 120/120 [01:59<00:00,  1.00it/s]

True label 1 Predicted label 1


In [91]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [92]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [93]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.59      0.58      0.59        60
           1       0.59      0.60      0.60        60

    accuracy                           0.59       120
   macro avg       0.59      0.59      0.59       120
weighted avg       0.59      0.59      0.59       120



## Test Set TESS

In [94]:
df_test = TESS_test

In [95]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)

In [96]:
predictions = make_predictions(df_test, labels,'2_12')

  0%|          | 0/1400 [00:00<?, ?it/s]

2_12


  0%|          | 1/1400 [00:00<17:10,  1.36it/s]

True label 1 Predicted label 0
2_12


  0%|          | 2/1400 [00:01<16:38,  1.40it/s]

...

True label 0 Predicted label 0
2_12


100%|█████████▉| 1399/1400 [16:50<00:00,  1.68it/s]

True label 0 Predicted label 1
2_12


100%|██████████| 1400/1400 [16:50<00:00,  1.39it/s]

True label 0 Predicted label 1


In [97]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [98]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [99]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.68      0.44      0.54       600
           1       0.67      0.84      0.75       800

    accuracy                           0.67      1400
   macro avg       0.68      0.64      0.64      1400
weighted avg       0.67      0.67      0.66      1400



## Test Set CREMA

In [100]:
df_test = CREMA_test

In [101]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)

In [102]:
predictions = make_predictions(df_test, labels,'2_12')

  0%|          | 0/60 [00:00<?, ?it/s]

2_12


  2%|▏         | 1/60 [00:00<00:35,  1.64it/s]

True label 1 Predicted label 1
2_12


  3%|▎         | 2/60 [00:01<00:40,  1.44it/s]

...

True label 0 Predicted label 0
2_12


 98%|█████████▊| 59/60 [00:36<00:00,  1.49it/s]

True label 1 Predicted label 1
2_12


100%|██████████| 60/60 [00:36<00:00,  1.64it/s]

True label 1 Predicted label 1


In [103]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [104]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [105]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.71      0.83      0.77        12
           1       0.96      0.92      0.94        48

    accuracy                           0.90        60
   macro avg       0.84      0.88      0.85        60
weighted avg       0.91      0.90      0.90        60

